$if you want to run the model go to the heading below called "you can diretly jump here and work on the model"

In [1]:
# loading the neccessary packages
import os
import numpy as np
import pandas as pd
import cv2 as cv
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import joblib
from imblearn.over_sampling import SMOTE

In [2]:
# creaing np.arrays by knowing the count of files of photo i have
age=np.zeros((9780,1))
gender=np.zeros((9780,1))

In [3]:
# getting age and gender which is available in filename
path="G:/complete data science/UTK datasets/crop_part1/"
count=0
for entry in os.listdir(path):
    age[count]=entry.split("_")[0]
    gender[count]=entry.split("_")[1]
    count+=1
count   

9780

In [4]:
# horizontal stacking for creating a dataframe
data=np.hstack((age,gender))
len(data)

9780

In [5]:
df=pd.DataFrame(data,columns=["age","gender"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9780 entries, 0 to 9779
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     9780 non-null   float64
 1   gender  9780 non-null   float64
dtypes: float64(2)
memory usage: 152.9 KB


In [6]:
# to cluster them into 5 groups other wise it has 100 diiferent ages which asks huge volume data instead ,we have only
# 9780 datas 
def changing_age(i):
    i = int(i)
    if i < 20:
        return 0
    elif (i>=20) and (i<40):
        return 1
    elif (i>=40) and (i<60):
        return 2
    elif (i>=60) and (i<80):
        return 3
    elif (i>=80) and (i<100):
        return 4
    else:
        return 5 

In [7]:
# making new columns
df["norm_age"]=df["age"].apply(changing_age)

In [8]:
df.gender.value_counts()

1.0    5407
0.0    4372
3.0       1
Name: gender, dtype: int64

df[df.gender==3.0].index

df.drop(df[df.gender==3.0].index,axis=0,inplace=True)

df.gender.value_counts()

In [11]:
# getting the image as one dimensional array ,here 9780 is nothing but the len(images) that we have in file folder 
arr=np.zeros((9780,32*32))
# sample to see how it would be?
arr[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [12]:
count=0
for entry in os.listdir(path):
#     print(path+entry)
    img=cv.imread(path+entry)
    gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    gray=cv.resize(gray,(32,32))
#     print(gray)
    arr[count]=gray.reshape(1,32*32)
#     print(gray.shape)
    count+=1
   

In [13]:
# normalize data to avoid skew in the datasets
arr = arr.astype('float32')
arr /= 255
arr

array([[0.9137255 , 0.45490196, 0.36078432, ..., 0.03137255, 0.12156863,
        0.27450982],
       [0.14117648, 0.13333334, 0.13725491, ..., 0.2627451 , 0.29803923,
        0.29803923],
       [0.8745098 , 0.88235295, 0.8745098 , ..., 0.3019608 , 0.25882354,
        0.22745098],
       ...,
       [0.8235294 , 0.78039217, 0.36078432, ..., 0.7411765 , 0.78431374,
        0.01568628],
       [0.03137255, 0.03529412, 0.05490196, ..., 0.14901961, 0.12941177,
        0.14901961],
       [0.72156864, 0.43529412, 0.25490198, ..., 0.06666667, 0.0627451 ,
        0.08627451]], dtype=float32)

In [14]:
# making dataframe 
df1=pd.DataFrame(arr)
df1

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.913725,0.454902,0.360784,0.619608,0.956863,0.913725,0.874510,0.933333,0.890196,0.905882,...,0.223529,0.164706,0.082353,0.066667,0.039216,0.015686,0.023529,0.031373,0.121569,0.274510
1,0.141176,0.133333,0.137255,0.105882,0.086275,0.164706,0.219608,0.266667,0.349020,0.403922,...,0.282353,0.117647,0.125490,0.172549,0.211765,0.262745,0.254902,0.262745,0.298039,0.298039
2,0.874510,0.882353,0.874510,0.803922,0.392157,0.796078,0.921569,0.945098,0.917647,0.858824,...,0.619608,0.709804,0.596078,0.627451,0.666667,0.654902,0.243137,0.301961,0.258824,0.227451
3,0.313726,0.278431,0.180392,0.196078,0.227451,0.250980,0.423529,0.454902,0.392157,0.313726,...,0.639216,0.658824,0.682353,0.713726,0.729412,0.733333,0.752941,0.709804,0.619608,0.235294
4,0.262745,0.207843,0.286275,0.509804,0.639216,0.666667,0.682353,0.713726,0.737255,0.752941,...,0.749020,0.721569,0.298039,0.203922,0.294118,0.101961,0.235294,0.411765,0.827451,0.690196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9775,0.996078,0.992157,0.988235,0.466667,0.301961,0.223529,0.627451,0.698039,0.737255,0.768627,...,0.329412,0.160784,0.196078,0.207843,0.215686,0.215686,0.196078,0.164706,0.164706,0.129412
9776,0.000000,0.003922,0.019608,0.015686,0.015686,0.003922,0.007843,0.043137,0.176471,0.274510,...,0.725490,0.490196,0.050980,0.011765,0.007843,0.023529,0.062745,0.035294,0.023529,0.015686
9777,0.823529,0.780392,0.360784,0.431373,0.439216,0.600000,0.749020,0.788235,0.807843,0.807843,...,0.654902,0.435294,0.235294,0.709804,0.698039,0.760784,0.760784,0.741176,0.784314,0.015686
9778,0.031373,0.035294,0.054902,0.160784,0.298039,0.372549,0.486275,0.580392,0.658824,0.721569,...,0.666667,0.705882,0.666667,0.584314,0.513726,0.380392,0.164706,0.149020,0.129412,0.149020


In [15]:
#we are creating csv so, we are in situation to create array of images every time we run the notebook
the_pandas_datframe=df1.to_csv("G:/csv_files/UTK_DATASETS_project.csv",index=False)

In [9]:
df1=pd.read_csv("G:/csv_files/UTK_DATASETS_project.csv")
df1

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.913725,0.454902,0.360784,0.619608,0.956863,0.913725,0.874510,0.933333,0.890196,0.905882,...,0.223529,0.164706,0.082353,0.066667,0.039216,0.015686,0.023529,0.031373,0.121569,0.274510
1,0.141176,0.133333,0.137255,0.105882,0.086275,0.164706,0.219608,0.266667,0.349020,0.403922,...,0.282353,0.117647,0.125490,0.172549,0.211765,0.262745,0.254902,0.262745,0.298039,0.298039
2,0.874510,0.882353,0.874510,0.803922,0.392157,0.796078,0.921569,0.945098,0.917647,0.858824,...,0.619608,0.709804,0.596078,0.627451,0.666667,0.654902,0.243137,0.301961,0.258824,0.227451
3,0.313725,0.278431,0.180392,0.196078,0.227451,0.250980,0.423529,0.454902,0.392157,0.313725,...,0.639216,0.658824,0.682353,0.713726,0.729412,0.733333,0.752941,0.709804,0.619608,0.235294
4,0.262745,0.207843,0.286275,0.509804,0.639216,0.666667,0.682353,0.713726,0.737255,0.752941,...,0.749020,0.721569,0.298039,0.203922,0.294118,0.101961,0.235294,0.411765,0.827451,0.690196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9775,0.996078,0.992157,0.988235,0.466667,0.301961,0.223529,0.627451,0.698039,0.737255,0.768627,...,0.329412,0.160784,0.196078,0.207843,0.215686,0.215686,0.196078,0.164706,0.164706,0.129412
9776,0.000000,0.003922,0.019608,0.015686,0.015686,0.003922,0.007843,0.043137,0.176471,0.274510,...,0.725490,0.490196,0.050980,0.011765,0.007843,0.023529,0.062745,0.035294,0.023529,0.015686
9777,0.823529,0.780392,0.360784,0.431373,0.439216,0.600000,0.749020,0.788235,0.807843,0.807843,...,0.654902,0.435294,0.235294,0.709804,0.698039,0.760784,0.760784,0.741177,0.784314,0.015686
9778,0.031373,0.035294,0.054902,0.160784,0.298039,0.372549,0.486274,0.580392,0.658824,0.721569,...,0.666667,0.705882,0.666667,0.584314,0.513725,0.380392,0.164706,0.149020,0.129412,0.149020


# TAKING AGE AS TARGET VARIABLE

# MAKING BALANCED DATASETS

In [10]:
df["norm_age"].unique()

array([5, 0, 1, 2, 3, 4], dtype=int64)

In [11]:
df["norm_age"].value_counts()

0    4164
1    2551
2    1613
3    1035
4     412
5       5
Name: norm_age, dtype: int64

In [12]:
deleting_index=df[(df["norm_age"]==5)].index
deleting_index

Int64Index([0, 1, 2, 143, 144], dtype='int64')

In [13]:
df1.drop(deleting_index,inplace=True)

In [14]:
df=df[~(df["norm_age"]==5)]

In [15]:
df["norm_age"].value_counts()

0    4164
1    2551
2    1613
3    1035
4     412
Name: norm_age, dtype: int64

In [16]:
smote=SMOTE( sampling_strategy="not majority")

In [17]:
x_sm,y_sm= smote.fit_resample(df1,df["norm_age"])

In [18]:
y_sm.value_counts()

0    4164
1    4164
2    4164
3    4164
4    4164
Name: norm_age, dtype: int64

In [19]:
x_train,x_test,y_train,y_test=train_test_split(x_sm,y_sm,test_size=0.3,random_state=5,stratify=y_sm)

In [20]:
best_clf=SVC(C=20,kernel="linear")

In [21]:
best_clf.fit(x_train,y_train)

SVC(C=20, kernel='linear')

In [23]:
best_clf.score(x_train,y_train)

0.9569781803211198

In [22]:
best_clf.score(x_test,y_test)

0.8474223503041947

# TAKING GENDER AS VARIABLE

In [24]:
df["gender"].unique()

array([0., 1., 3.])

In [25]:
df["gender"].value_counts()

1.0    5402
0.0    4372
3.0       1
Name: gender, dtype: int64

In [26]:
df[df["gender"]==3.0].index

Int64Index([7884], dtype='int64')

In [27]:
df.drop([7884],axis=0,inplace=True)

In [31]:
df1.drop([7884],axis=0,inplace=True)

In [32]:
df["gender"].value_counts()

1.0    5402
0.0    4372
Name: gender, dtype: int64

In [29]:
smote=SMOTE( sampling_strategy="minority")

In [33]:
x_sm,y_sm= smote.fit_resample(df1,df["gender"])
y_sm.value_counts()

0.0    5402
1.0    5402
Name: gender, dtype: int64

In [40]:
x_train,x_test,y_train,y_test=train_test_split(x_sm,y_sm,test_size=0.3,random_state=5, stratify=y_sm)

model 1

In [41]:
best_clf_gender=RandomForestClassifier(n_estimators=100)
best_clf_gender.fit(x_train,y_train)

RandomForestClassifier()

In [42]:
best_clf_gender.score(x_train,y_train)

0.9993387992594551

In [43]:
best_clf_gender.score(x_test,y_test)

0.7757557063541024

model 2

In [45]:
best_clf_gender=RandomForestClassifier(n_estimators=1000)
best_clf_gender.fit(x_train,y_train)

RandomForestClassifier(n_estimators=1000)

In [46]:
best_clf_gender.score(x_train,y_train)

0.9993387992594551

In [47]:
best_clf_gender.score(x_test,y_test)

0.784083898827884

model 3

In [48]:
best_clf_gender3=SVC(C=20,kernel="linear")
best_clf_gender3.fit(x_train,y_train)

SVC(C=20, kernel='linear')

In [49]:
best_clf_gender3.score(x_train,y_train)

0.8680243321872521

In [50]:
best_clf_gender3.score(x_test,y_test)

0.7714373843306601

#chosing RandomForestClassifier(n_estimators=1000) as best model

#  you can hyper tune your model with dofferent params and different algorithms and can get better results

In [60]:
# saving the model 
filename1 = 'UTK_DATASETS_age_MODEL.pkl'
filename2 = 'UTK_DATASETS_gender_MODEL.pkl'
joblib.dump(best_clf, filename1)
joblib.dump(best_clf_gender, filename2)

['UTK_DATASETS_gender_MODEL.pkl']

# you can diretly jump here and work on the model

In [2]:
age_predictor=joblib.load("G:/jupyter notebook/UTK_DATASETS_age_MODEL.pkl")
gender_predictor=joblib.load("G:/jupyter notebook/UTK_DATASETS_gender_MODEL.pkl")

give the file path as input (better give single face photo)

In [71]:
face_cascade = cv.CascadeClassifier('C:/Users/Hariharan.M/AppData/Roaming/Python/Python310/site-packages/cv2/data/haarcascade_frontalface_default.xml')
eye_cascade = cv.CascadeClassifier('C:/Users/Hariharan.M/AppData/Roaming/Python/Python310/site-packages/cv2/data/haarcascade_eye.xml')

dic={0:"0-20",1:"20-40",2:"40-60",3:"60-80",4:"80-100",5:"100-120"}   #for age purpose
d={"[0.]":"male","[1.]":"female"}   # for gender purpose

In [117]:
def age_sex_predictor(string):
    frame=cv.imread(string)
    gray=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 2)
    for i in faces:
        try:
            x,y,w,h=faces[i]
            break
        except:
             pass
    else:
        print("cannot find you face")
        return
    rectangle_img = cv.rectangle(gray,(x,y),(x+w,y+h),(255,0,0),2) 
    face_img=frame[y:y+h,x:x+w]
    gray=cv.cvtColor(face_img,cv.COLOR_BGR2GRAY)
    gray=cv.resize(gray,(32,32))
    reshape_face=gray.reshape(1,32*32)
    arr = reshape_face.astype('float32')
    arr /= 255
    gender_predict=gender_predictor.predict(arr)
    gender_predict=str(gender_predict)
    age_predict=age_predictor.predict(arr)
    age_predict=age_predict[0]
    print("hey!,there you are ",d[gender_predict])
    print("your age is ",dic[age_predict])
    return face_img

In [93]:
# give the file path as string
string="" #please give the file path    
age_sex_predictor(string)

C:\Users\Hariharan.M\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\Hariharan.M\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


hey!,there you are  female
your age is  40-60


In [95]:
import matplotlib.pyplot as plt

In [118]:
# give the file path as string
hhh=age_sex_predictor("G:/UTKFace/100_0_0_20170112215240346.jpg.chip.jpg")
# plt.imshow(hhh)

cannot find you face


# USING WEBCAM

In [10]:
import warnings
warnings.simplefilter("ignore", UserWarning)

In [3]:
face_cascade = cv.CascadeClassifier('C:/Users/Hariharan.M/AppData/Roaming/Python/Python310/site-packages/cv2/data/haarcascade_frontalface_default.xml')
eye_cascade = cv.CascadeClassifier('C:/Users/Hariharan.M/AppData/Roaming/Python/Python310/site-packages/cv2/data/haarcascade_eye.xml')

dic={0:"0-20",1:"20-40",2:"40-60",3:"60-80",4:"80-100",5:"100-120"}

In [11]:
cap=cv.VideoCapture(0)
i=0
while True:
    ret,frame=cap.read()
    gray=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 2)
    try:
        x,y,w,h=faces[0]
    except:
        cv.imshow("age&gender_detector",frame)
        if cv.waitKey(1)==ord("q"):
            break
        continue
    rectangle_img = cv.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2) 
    face_img=frame[y:y+h,x:x+w]
    gray=cv.cvtColor(face_img,cv.COLOR_BGR2GRAY)
    gray=cv.resize(gray,(32,32))
    reshape_face=gray.reshape(1,32*32)
    arr = reshape_face.astype('float32')
    arr /= 255
    gender=gender_predictor.predict(arr)
    gender=str(gender)
    your_age=age_predictor.predict(arr)
    your_age=your_age[0]
    if gender=="[0.]":
        txt_img=cv.putText( rectangle_img,f"you are male {dic[your_age]}",(x,y),cv.FONT_HERSHEY_PLAIN,1.0,(0,255,0),thickness=2)
    else:
        txt_img=cv.putText( rectangle_img,f"you are female {dic[your_age]}",(x,y),cv.FONT_HERSHEY_PLAIN,1.0,(0,255,0),thickness=2)
    cv.imshow("age&gender_detector", txt_img)
    if cv.waitKey(1)==ord("q"):
        break
cap.release() 
cv.destroyAllWindows()
